In [ ]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [ ]:
!pip install tensorflow==2.15.0
!pip install tensorflow-addons --upgrade

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer
import shutil
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [7]:
def split_and_save_dataset(input_dir, output_dir, train_ratio=0.7, val_ratio=0.15):
    """
    Membagi dataset menjadi training, validation, dan testing.
    - input_dir: Folder asal dataset.
    - output_dir: Folder tempat dataset hasil split akan disimpan.
    - train_ratio: Rasio data untuk training.
    - val_ratio: Rasio data untuk validation (sisanya untuk testing).
    """
    # Hitung rasio untuk split
    test_ratio = 1 - train_ratio - val_ratio

    # Buat struktur folder output
    for split in ['training', 'validation', 'testing']:
        split_path = os.path.join(output_dir, split)
        os.makedirs(split_path, exist_ok=True)

    # Iterasi setiap kelas di folder input
    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)

        if not os.path.isdir(class_path):
            continue

        # Dapatkan semua file dalam folder kelas
        files = [os.path.join(class_path, f) for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

        # Split dataset
        train_files, temp_files = train_test_split(files, train_size=train_ratio, random_state=42)
        val_files, test_files = train_test_split(temp_files, test_size=test_ratio / (test_ratio + val_ratio), random_state=42)

        # Fungsi untuk memindahkan file ke folder
        def move_files(file_list, split):
            split_class_path = os.path.join(output_dir, split, class_name)
            os.makedirs(split_class_path, exist_ok=True)
            for file in file_list:
                shutil.copy(file, os.path.join(split_class_path, os.path.basename(file)))

        # Pindahkan file ke folder masing-masing
        move_files(train_files, 'training')
        move_files(val_files, 'validation')
        move_files(test_files, 'testing')

# Path ke dataset awal dan folder hasil split
input_dir = '/content/drive/MyDrive/asl_dataset'  # Folder dataset awal
output_dir = '/content/splitt_asl_dataset'     # Folder hasil split

split_and_save_dataset(input_dir, output_dir)

In [9]:
# validasi hasil setelah splitting
def count_files_in_split(output_dir):
    for split in ['training', 'validation', 'testing']:
        split_path = os.path.join(output_dir, split)
        total_files = sum([len(files) for _, _, files in os.walk(split_path)])
        print(f"{split.capitalize()} files: {total_files}")

count_files_in_split('/content/splitt_asl_dataset')

Training files: 1760
Validation files: 359
Testing files: 396


In [15]:
dataset_path = "/content/splitt_asl_dataset/training"
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, i)
    if os.path.isdir(folder_path):  # Pastikan hanya folder yang ditambahkan
        labels.append(i)
print(labels)

/content/splitt_asl_dataset/training
['0', '6', 'f', 'b', 'v', '4', 'c', 'none', '7', '1', 'd', 'm', 'y', '2', '9', 'u', '3', '8', '5', 'x', 'l', 'n', 'a', 'o', 'h', 'k', 'q', 'g', 'p', 'e', 'j', 'r', 't', 'w', 's', 'z', 'i']


In [16]:
# cek apakah folder "None" ada dan hapus
none_folder_path = os.path.join(dataset_path, "none")
if os.path.exists(none_folder_path) and os.path.isdir(none_folder_path):
    shutil.rmtree(none_folder_path)
    print(f"Folder 'none' berhasil dihapus.")
else:
    print("Folder 'none' tidak ditemukan.")

Folder 'none' berhasil dihapus.


In [25]:
# Tentukan path ke folder dataset, misalnya ke folder "training"
dataset_path = "/content/splitt_asl_dataset/training"

# Buat folder 'None' jika belum ada
none_folder_path = os.path.join(dataset_path, "None")
if not os.path.exists(none_folder_path):
    os.makedirs(none_folder_path)
    print(f"Folder 'None' berhasil dibuat di {none_folder_path}")

# Memuat dataset dengan preprocessing yang sesuai
dataset = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,  # Pastikan path ini sesuai dengan folder 'training', 'validation', atau 'testing'
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder


In [26]:
# data = gesture_recognizer.Dataset.from_folder(
#     dirname=dataset_path,
#     hparams=gesture_recognizer.HandDataPreprocessingParams()
# )

In [28]:
# dataset path 2
dataset_path_2 = "/content/splitt_asl_dataset/validation"

# Buat folder 'None' jika belum ada
none_folder_path = os.path.join(dataset_path_2, "None")
if not os.path.exists(none_folder_path):
    os.makedirs(none_folder_path)
    print(f"Folder 'None' berhasil dibuat di {none_folder_path}")

Folder 'None' berhasil dibuat di /content/splitt_asl_dataset/validation/None


In [29]:
hparams = gesture_recognizer.HParams(learning_rate=0.001, epochs=100, export_dir="exported_model_1")
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.2)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model_2 = gesture_recognizer.GestureRecognizer.create(
    train_data=gesture_recognizer.Dataset.from_folder(dirname="/content/splitt_asl_dataset/training"),
    validation_data=gesture_recognizer.Dataset.from_folder(dirname="/content/splitt_asl_dataset/validation"),
    options=options
)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder
Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                       

In [30]:
model_2.export_model()
!ls exported_model_1

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
best_model_weights.data-00000-of-00001	checkpoint    gesture_recognizer.task  metadata.json
best_model_weights.index		epoch_models  logs


In [32]:
files.download('exported_model_1/gesture_recognizer.task')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>